We built and deployed a news recommendation system based on user and news embeddings in [part 2](https://blog.vespa.ai/build-news-recommendation-app-from-python-with-vespa/) of this tutorial series. The embeddings were trained using the model depicted in the figure below and described in detail [here](https://docs.vespa.ai/en/tutorials/news-4-embeddings.html). In this tutorial we show how to check if the embeddings deployed in the recommendation system are working as expected. Checking evaluation metrics of the final search application is at least as important as cheking evaluation metrics when training a model. We want `pyvespa` to make this process as easy as possible.

![Embedding's model](data/2021-04-26-evaluation-news-recommendation/embeddings.png)

When training the model we observed the following training and evaluation metrics over the course of 10 epochs.

In [ ]:
$ python3 src/python/train_cold_start.py mind 10
Total loss after epoch 1: 920.5855102539062 (0.703811526298523 avg)
{'auc': 0.5391, 'mrr': 0.2367, 'ndcg@5': 0.2464, 'ndcg@10': 0.3059}
{'auc': 0.5131, 'mrr': 0.2239, 'ndcg@5': 0.2296, 'ndcg@10': 0.2933}
Total loss after epoch 2: 761.7719116210938 (0.5823944211006165 avg)
{'auc': 0.647, 'mrr': 0.2992, 'ndcg@5': 0.3246, 'ndcg@10': 0.3829}
{'auc': 0.5656, 'mrr': 0.2447, 'ndcg@5': 0.2604, 'ndcg@10': 0.3255}
...
Total loss after epoch 10: 517.16748046875 (0.3953879773616791 avg)
{'auc': 0.8758, 'mrr': 0.5074, 'ndcg@5': 0.5818, 'ndcg@10': 0.6316}
{'auc': 0.6249, 'mrr': 0.2842, 'ndcg@5': 0.3114, 'ndcg@10': 0.3733}

Once we have deployed the embeddings in our recommendation system, we should check if we can recover similar evaluation metrics when sending the appropriate queries to the application.

In [2]:
%config Completer.use_jedi = False

In [10]:
import io, csv

def read_impressions_file(file_name):
    impressions = []
    if not os.path.exists(file_name):
        print("{} not found.".format(file_name))
        sys.exit(1)
    print("Reading impressions data from " + file_name)

    with io.open(file_name, "r", encoding="utf-8") as f:
        field_list = ["id", "user_id", "timestamp", "history", "impressions"]
        reader = csv.DictReader(f, delimiter="\t", fieldnames=field_list)
        for line in reader:
            user_id = line["user_id"]
            impression = {
                "user_id": user_id,
                "news_ids": [],
                "labels": []
            }
            for i in line["impressions"].split(" "):
                news_id, label = i.split("-")
                impression["news_ids"].append(news_id)
                impression["labels"].append(int(label))
            impressions.append(impression)
    return impressions

In [6]:
import os

data_dir = "data/2021-03-02-news/mind/"
train_impressions_file = os.path.join(data_dir, "train", "behaviors.tsv")
valid_impressions_file = os.path.join(data_dir, "dev", "behaviors.tsv")

In [15]:
train_impressions = read_impressions_file(train_impressions_file)
valid_impressions = read_impressions_file(valid_impressions_file)

Reading impressions data from data/2021-03-02-news/mind/train/behaviors.tsv
Reading impressions data from data/2021-03-02-news/mind/dev/behaviors.tsv


In [14]:
train_impressions[1]

{'user_id': 'U84185',
 'news_ids': ['N13089',
  'N18101',
  'N1248',
  'N26273',
  'N12770',
  'N1132',
  'N13649'],
 'labels': [0, 0, 0, 0, 1, 0, 0]}